In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving images.zip to images.zip
User uploaded file "images.zip" with length 7207140 bytes


In [ ]:
import zipfile
import os


zip_file_path = "images.zip"

# Extracting the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall("/content/dataset")

Preprocessing the data and Split the data in to Train and Test

In [ ]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

# Define transformations
transform = transforms.Compose([
    transforms.Resize((32, 32)),  # Resize images to 32x32
    transforms.ToTensor(),         # Convert images to tensors
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize images
])

# Path to the extracted dataset
dataset_path = "/content/dataset/images"

# Load the training dataset
train_dataset = ImageFolder(root=dataset_path + "/train", transform=transform)

# Load the testing dataset
test_dataset = ImageFolder(root=dataset_path + "/test", transform=transform)


Train and evaluate the sklearn Classifier

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Extract features and labels from the datasets
train_features = [sample[0].numpy().flatten() for sample in train_dataset]
train_labels = [sample[1] for sample in train_dataset]

test_features = [sample[0].numpy().flatten() for sample in test_dataset]
test_labels = [sample[1] for sample in test_dataset]

# Initialize the Support Vector Classifier (SVC)
classifier = SVC()

# Train the classifier
classifier.fit(train_features, train_labels)

# Make predictions on the test data
predictions = classifier.predict(test_features)

# Calculate accuracy
accuracy = accuracy_score(test_labels, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.755


Train and Evaluate a Deep Learning Model

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
dataset_path = "/content/dataset/images"
# Define transformations
train_datagen = ImageDataGenerator(
    rotation_range=10,
    horizontal_flip=True,
    zoom_range=0.1,
    rescale=1./255,
    validation_split=0.2
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    dataset_path + "/train",
    target_size=(32, 32),
    batch_size=128,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    dataset_path + "/train",
    target_size=(32, 32),
    batch_size=128,
    class_mode='categorical',
    subset='validation'
)

test_generator = test_datagen.flow_from_directory(
    dataset_path + "/test",
    target_size=(32, 32),
    batch_size=128,
    class_mode='categorical'
)

# Define the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dropout(0.25),
    layers.Dense(128, activation='relu'),
    layers.Dense(100, activation='softmax')
])

# Compile the model
model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']
)


# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 128,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // 128
)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
print("Test Accuracy:", test_acc)


Found 8040 images belonging to 100 classes.
Found 1960 images belonging to 100 classes.
Found 1000 images belonging to 100 classes.
Epoch 1/10
62/62 [==============================] - 26s 381ms/step - loss: 4.3738 - accuracy: 0.0466 - val_loss: 4.4010 - val_accuracy: 0.0682
Epoch 2/10
62/62 [==============================] - 26s 418ms/step - loss: 3.1388 - accuracy: 0.2227 - val_loss: 4.7803 - val_accuracy: 0.0375
Epoch 3/10
62/62 [==============================] - 26s 411ms/step - loss: 1.9509 - accuracy: 0.4532 - val_loss: 5.7711 - val_accuracy: 0.0240
Epoch 4/10
62/62 [==============================] - 26s 410ms/step - loss: 1.3312 - accuracy: 0.6145 - val_loss: 5.9748 - val_accuracy: 0.0292
Epoch 5/10
62/62 [==============================] - 24s 388ms/step - loss: 1.0094 - accuracy: 0.7104 - val_loss: 5.0954 - val_accuracy: 0.0604
Epoch 6/10
62/62 [==============================] - 24s 388ms/step - loss: 0.8131 - accuracy: 0.7652 - val_loss: 3.7032 - val_accuracy: 0.1464
Epoch 7/10